In [3]:
import gym

from stable_baselines import GAIL, SAC
from stable_baselines.gail import ExpertDataset, generate_expert_traj

# Generate expert trajectories (train expert)
model = SAC('MlpPolicy', 'Pendulum-v0', verbose=1)
generate_expert_traj(model, 'expert_pendulum', n_timesteps=100, n_episodes=10)

# Load the expert dataset
dataset = ExpertDataset(expert_path='expert_pendulum.npz', traj_limitation=10, verbose=1)

model = GAIL('MlpPolicy', 'Pendulum-v0', dataset, verbose=1)
# Note: in practice, you need to train for 1M steps to have a working policy
model.learn(total_timesteps=1000)
model.save("gail_pendulum")

del model # remove to demonstrate saving and loading

model = GAIL.load("gail_pendulum")

env = gym.make('Pendulum-v0')
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()

Creating environment from the given name, wrapped in a DummyVecEnv.




Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.Dense instead.




/usr/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




actions (2000, 1)
obs (2000, 3)
rewards (2000,)
episode_returns (10,)
episode_starts (2000,)
actions (2000, 1)
obs (2000, 3)
rewards (2000,)
episode_returns (10,)
episode_starts (2000,)
Total trajectories: 10
Total transitions: 2000
Average returns: -1578.0879550158977
Std for returns: 234.12349211845012
Creating environment from the given name, wrapped in a DummyVecEnv.




********** Iteration 0 ************
Optimizing Policy...
sampling

done in 1.533 seconds
computegrad

done in 0.160 seconds
conjugate_gradient
      iter residual norm  soln norm
         0     0.0566          0
         1    0.00202     0.0185
         2   0.000205      0.034
         3   0.000198     0.0509
         4    0.00018      0.174
         5   0.000244      0.234
         6     0.0036      0.284
         7    9.2e-05      0.385
         8   6.66e-06      0.402
         9    5.4e-07      0.464
        10 

/usr/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


ImportError: 
    Error occurred while running `from pyglet.gl import *`
    HINT: make sure you have OpenGL install. On Ubuntu, you can run 'apt-get install python-opengl'.
    If you're running on a server, you may need a virtual frame buffer; something like this should work:
    'xvfb-run -s "-screen 0 1400x900x24" python <your_script.py>'
    

In [4]:
import sys
import numpy as np
import tensorflow as tf
from datetime import datetime

device_name = 'cpu' # Choose device from cmd line. Options: gpu or cpu
shape = (int(sys.argv[2]), int(sys.argv[2]))
if device_name == "gpu":
    device_name = "/gpu:0"
else:
    device_name = "/cpu:0"

with tf.device(device_name):
    random_matrix = tf.random_uniform(shape=shape, minval=0, maxval=1)
    dot_operation = tf.matmul(random_matrix, tf.transpose(random_matrix))
    sum_operation = tf.reduce_sum(dot_operation)

startTime = datetime.now()
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as session:
        result = session.run(sum_operation)
        print(result)

# It can be hard to see the results on the terminal with lots of output -- add some newlines to improve readability.
print("\n" * 5)
print("Shape:", shape, "Device:", device_name)
print("Time taken:", str(datetime.now() - startTime))

ValueError: invalid literal for int() with base 10: '/root/.local/share/jupyter/runtime/kernel-26ec60c9-625c-4248-a9b1-ff5fd7a3263a.json'